Goal: Create a program to study the consequences of establishing habitat corridors between habitat patches

Your framework will have three classes: Individual, Population, and Landscape. Individuals exist in populations and populations exist in landscapes.

In every time step, individuals can stay in the population where they started or move to a new population, the probability of which is stored in a dispersal matrix.

Individuals need to have: phenotype attribute

Populations need: list of individuals
    Constructor to create new list for ind in pop
    Methods to add and remove ind
    Calc and print the freq of phenotypes among ind
    
Lanscapes need: list of populations, dispersal matrix
    Constructor to create new list for populations in that landscape
    Method to use probabilities in dispersal matrix to determine if ind moves or stays each time

Dispersal matrix - consists of the probabilities that an ind moves populations or stays in place each time step. 
    For this species (Warbling Babblers) this corresponds to the possibility of moving in a week
    If an ind moves pops, remove it from the list of ind in the pop it started in and add it to the list in the pop it goes to
        Do not modify a list that you are looping through inside of the loop itself, do it once the loop is complete
    
Notes:
    Population or landscape constructors should set the starting population sizes
    Total number of ind across all populations should stay constant, but ind pops change sizes
    
When simulation begins, all ind in a population should have the same phenotype. As ind move between pop through time steps (weeks), track the trajectory of phenotype freq and pop sizes
    Will need to write a function to create these plots, either as a method of one of the classes or independent 
    Can overlay pt freq or pop sizes for diff pop on a single plt for easy comparison 

In [1]:
#Import any needed modules
import random
import matplotlib.pyplot as plt
import numpy as np

Individual class notes:
Needs at minimum a phenotype attribute

In [2]:
#Create individual class, with each individual having a phenotype attribute
class individual:
    """Class to hold information about individuals"""
    
    #Create constructor to construct individuals
    def __init__(self,id,pt):
        """Used to create individuals and assign them an id and phenotype(pt)"""
        
        self.id = id
        self.pt = pt

Population class notes:
Needs at minimum a list of individuals

In [3]:
#Create population class
class population:
    """Class to hold information about populations"""
    
    #Create constructor to create lists of individuals in the population, defaulting to a population size of 10 and a brown phenotype
    def __init__(self,pt="brown",popSize=10,popnum=1):
        """Creates list of individuals in the population"""
        
        self.popSize = popSize
        self.popnum = popnum
        
        #Create list of individuals in each population (0 and 1)
        self.indPop = []
        
        #Create individuals and add them to the population list
        for num in range(popSize):
            self.indPop.append(individal(id=num,pt=pt))
    

    #Method to calculate and print the frequency of phenotypes in the population
    def calcPtFreq(self):
        """Calculates the frequency of phenotypes in the population"""
                        
        #Update population size
        totalPop = count(ind)
        #Save that in a list to use to make the plot later
        #popSizeCount.append(totalPop)
        #Calculate the number of ind with the brown phenotype
        if pt == "brown":
            bn=count(pt)
            #Calculate the frequenct of the brown phenotype in the population
            bnf = bn / self.popSize
            #Save in list
            #bnfCount.append(bnf)
        #Calculate the white phenotype frequency using the same method
        if pt == "white":
            wt=count(pt)
            wtf = wt / self.popSize
            #wtfCount.append(wtf)
        
        #Create data list to return in a bit
        data = []
        data.append(totalPop,bnf,wtf)
        
        #Return the data to use later 
        return data
        


Landscape class notes:
Need at minimum a list of populations and a dispersal matrix

Dispersal matrix stores probabilities that an individual disperses or stays in place
    May be useful to use the random.choices


In [4]:
#Create landscape class
class landscape:
    """Class to hold information about landscapes"""
    
    #Create constructor to create list of populations in that landscape, and assign each population an id
    def __init__(self,pt="brown",id=1,popnum=1):
    
        self.id = id
        self.popnum = popnum

        #create list of populations in that landscape
        self.popList = []
        
        #Create the dispersal matrix, with a .8 chance ind in pop1 will not move, a a.2 chance ind in pop1 will move to pop2
        #a .7 chance ind in pop2 will not move, and a .3 chance ind in pop2 will move to pop1
        self. dispMatrix ={[0.8,0.2],
                           [0.3,0.7]}

        #Set phenotype
        if popnum == 1:
            pt = "white"
        else:
            pt = "brown"
        
        #Create populations and add them to the list
        for num in range(popnum):
            self.popList.append(population(id=num))
        
    #methods to use the dispersal matrix to move individuals
    def move():
        """Uses the probability matrix to move inidividuals between populations"""
        
        #Create queues to add and remove ind
        movefrom = []
        removefrom = []
        
        #Loop through each population
        for i in range(len(self.popList)):
            pop=self.popList[i]
            #Loop through each individual in the population
            for ind in pop.individuals:
                #Use the probability matrix and the random choices module to create new population
                newPop = random.choices(self.popList,self.dispMatrix[i])
                #Add individuals that move to queue to be moved to new pop and removed from their original population
                if newPop != pop:
                    movefrom[i].append(ind)
                    removefrom[i].append(ind)
        #Loop through each population
        for pop in self.popList:
            #If there is an individual that is in both the removefrom list and the population, remove it
            for ind in removefrom:
                if removefrom == pop:
                    pop.remove(ind)
            #If there is an individual in the move list that is not in the population, add it
            for ind in movefrom:
                if movefrom != pop:
                    pop.append(ind)
                    
        #Call on calcPtFreq method to get all data together
        calcPtFreq()
        
    
    def makePlots(info):
        """Method for making plots of phenotype frequency and population size change over time"""
    
        for i in popnum:
            if i == 0:
                info = data0
            if i == 1:
                info = data1
        
        #Create figure for plots
        plt.figure()
        
        #Create plot for frequency of the brown phenotype
        plt.subplot(131)
        y_1 = data0[1]
        y_2 = data1[1]    
        plt.plot(y_1,'^',markersize=3)
        plt.plot(y_2,'o',markersize=3)
        plt.ylabel("Frequency of the brown phenotype")
        plt.xlabel("Weeks")
        
        #Create plot for frequency of the white phenotype over time
        plt.subplot(132)
        y_3 = data0[2]
        y_4 = data1[2]
        plt.plot(y_3,'^',markersize=3)
        plt.plot(y_4,'o',markersize=3)
        plt.ylabel("Frequency of the white phenotype")
        plt.xlabel("Weeks")

        #create plot for population size over time
        plt.subplot(133)
        y_5 = data0[0]
        y_6 = data1[0]
        plt.plot(y_5,'^',markersize=3)
        plt.plot(y_6,'o',markersize=3)
        plt.ylabel("Population Size")
        plt.xlabel("Weeks")


Create plots of phenotype frequency and population size over time (2 pops on same plot to save time)

In [5]:
#Run code

#Create 2 populations of 10 individuals each, one with brown phenotype and one with white phenotype
OurLandscape = landscape()

#Run probabilities of moving for 10 weeks for both populations in the landscape we created
for num in range (1,10):
    OrLandscape.probMove()
    #The probMove method calls on the calcPtFreq method so the ind will move according to the probabilities
    
#Make plots
OurLandscape.makePlots(data)

TypeError: unhashable type: 'list'

QUESTIONS and SIMULATIONS:
1. How do the frequencies of phenotypes change in each population week-by-week as individuals move?
2. What effect does an overall increase in the rate of movement (migration) have?
3. What happens to both phenotype frequencies and population sizes when movement probabilities are not symmetric (individuals have a higher probability of moving from population A to population B, than they do of moving from population B to population A)?
4. What effect does changing the starting population sizes have on the trajectory of phenotype frequencies? In other words, what might happen if you connect big habitat patches versus small habitat patches?